# HV19.17 Unicode Portal

*Buy your special gifts online, but for the ultimative gift you have to become admin.*

In this challenge, we get a web portal focused on unicode characters:

![Portal screenshot](./portal.png)

After registering an account and logging in, we can access a "Source" page: *Since Santa is a big fan of open source software, he publicly provides his authentication code. For free!*.

From there, we can get this PHP snippet:

```php
<?php

if (isset($_GET['show'])) highlight_file(__FILE__);

/**
 * Verifies user credentials.
 */
function verifyCreds($conn, $username, $password) {
  $usr = $conn->real_escape_string($username);
  $res = $conn->query("SELECT password FROM users WHERE username='".$usr."'");
  $row = $res->fetch_assoc();
  if ($row) {
    if (password_verify($password, $row['password'])) return true;
    else addFailedLoginAttempt($conn, $_SERVER['REMOTE_ADDR']);
  }
  return false;
}

/**
 * Determines if the given user is admin.
 */
function isAdmin($username) {
  return ($username === 'santa');
}

/**
 * Determines if the given username is already taken.
 */
function isUsernameAvailable($conn, $username) {
  $usr = $conn->real_escape_string($username);
  $res = $conn->query("SELECT COUNT(*) AS cnt FROM users WHERE LOWER(username) = BINARY LOWER('".$usr."')");
  $row = $res->fetch_assoc();
  return (int)$row['cnt'] === 0;
}

/**
 * Registers a new user.
 */
function registerUser($conn, $username, $password) {
  $usr = $conn->real_escape_string($username);
  $pwd = password_hash($password, PASSWORD_DEFAULT);
  $conn->query("INSERT INTO users (username, password) VALUES (UPPER('".$usr."'),'".$pwd."') ON DUPLICATE KEY UPDATE password='".$pwd."'");
}

/**
 * Adds a failed login attempt for the given ip address. An ip address gets blacklisted for 15 minutes if there are more than 3 failed login attempts.
 */
function addFailedLoginAttempt($conn, $ip) {
  $ip = $conn->real_escape_string($ip);
  $conn->query("INSERT INTO fails (ip) VALUES ('".$ip."')");
}

?>
```

I can't see any SQL injection or obvious PHP issues (like using `==` instead of `===`) there, but there are some things that seem strange:

- We can access the "admin" page (which probably gives us the flag) if we can log in as the "santa" user.
- If `registerUser` gets called with a duplicate key, it updates the password of santa.
- The `registerUser` function calls `UPPER(...)` on the username
- However, we're prevented from registering an existing user due to the check in `isUsernameAvailable`
- Note that that function uses `LOWER(...) = BINARY LOWER(...)` to check for usernames.

At that point I was pretty sure this was related to case handling somehow. For `a-z`/`A-Z`, the mapping from lower- to uppercase is quite obvious. However, the challenge makes it obvious that we'll need to deal with Unicode somehow. In alphabets other than the typical English/latin alphabet, things can get hairy.

The typical example of this is probably [Turkish](http://www.i18nguy.com/unicode/turkish-i18n.html): Their alphabet has lower-/uppercase version of a dotless i (ı/I) and a dotted i (i/İ). Those are different sounds - in other words, upper/lowercasing words in a non locale-aware way results in a wrong word:

In [1]:
"Kadın".upper().lower()

'kadin'

This issue recently caused a [security bug](https://eng.getwisdom.io/hacking-github-with-unicode-dotless-i/) on GitHub! We can get something similar with the German ß as well:

In [2]:
"Straße".upper().lower()

'strasse'

To change the password of santa, we'll need to find something that passes as "santa" when registering (thus changing the password of the santa user), but doesn't get detected by `isUsernameAvailable`.

Since I wasn't exactly sure how those casing collisions work in practice, I started by reading the [Unicode case mapping FAQ](http://unicode.org/faq/casemap_charprop.html). There, a question caught my attention:

> Q: Why is there no unique uppercase character for ſ — U+017F LATIN SMALL LETTER LONG S (and about one hundred other characters)?

The [history of this character](https://en.wikipedia.org/wiki/Long_s) was quite an interesting read for the linguistics/typography nerd in me!

Anyways - that character sounds like something we could use to create a fake santa user:

In [3]:
"ſanta".upper()

'SANTA'

And indeed, after registering a "ſanta" user, we can then log in as "santa" with the password we selected. Then, we can access the admin page and get the flag, `HV19{h4v1ng_fun_w1th_un1c0d3}`.